# Metody Obliczeniowe w Nauce i Technice Laboratorium 5
## Równania nieliniowe
### Błażej Kustra

In [125]:
import matplotlib.pyplot as plt
import numpy as np
import random
import time

### 1. Metoda Gaussa-Jordana
Napisz i sprawdź funkcję rozwiązującą układ równań liniowych n × n metodą Gaussa-Jordana. Dla rozmiarów macierzy współczynników większych niż 500 × 500 porównaj czasy działania zaimplementowanej funkcji z czasami uzyskanymi dla wybranych funkcji bibliotecznych.

In [140]:
def create_linear_equation(n):
    M = np.random.randint(low=-100, high=100, size=(n, n)).astype(float)
    W = np.random.randint(low=1, high=100, size=n).astype(float)
    return M, W

In [158]:
def gauss_jordan(M, W):
    n = len(W)
    
    for i in range(n):   
        for j in range(i+1,n):
            if abs(M[j][i]) > abs(M[i][i]):
                M[[i, j]] = M[[j, i]] 
                W[i], W[j] = W[j], W[i]

    for i in range(n):
        scale = M[i][i]
        M[i] /= scale
        W[i] /= scale
        
        for j in range(n):
            if i == j: continue 
            scale = M[j][i] 
            M[j] -= M[i] * scale 
            W[j] -= W[i] * scale 
            
    return W

In [161]:
M, W = create_linear_equation(3)
print(M)
print(W)
print("linalg.solve: ", np.linalg.solve(M, W))
print("Gauss-Jordan: ", gauss_jordan(M, W))


[[ -6. -29.  82.]
 [ 86. -65.  27.]
 [-95.  19.  69.]]
[34. 36. 53.]
linalg.solve:  [-2.07692147 -3.7424481  -1.06088444]
Gauss-Jordan:  [-2.07692147 -3.7424481  -1.06088444]


In [128]:
def measure_time(M, W):
    start_np = time.time()
    np.linalg.solve(M, W)
    end_np = time.time()
    
    start_gauss = time.time()
    gauss_jordan(M, W)
    end_gauss = time.time()
    
    return end_np - start_np, end_gauss - start_gauss

In [133]:
time_np = []
time_gauss = []

for n in range(600):
    M, W = create_linear_equation(n)
    time_1, time_2 = measure_time(M, W)
    time_np.append(time_1)
    time_gauss.append(time_2)
    print(n, end="\r")

plt.plot(time_np,'r+')
plt.plot(time_gauss,'b+')
plt.show()    

Czas potrzebny do rozwiązania układu równań jest znacznie dłuższy niż funkcji $numpy.linalg.solve$, a różnica rośnie wraz ze wzrostem ilości elementów w macierzy. 

<img src="gauss-vs-linalg-solve.png">
<center> zaimplementowany program (niebieski) </center>
<center> funkcja np.linalg.solve (czerwony) </center>

### 2. Faktoryzacja LU
Napisz i sprawdź funkcję dokonującą faktoryzacji $A = LU$ macierzy $A$. Zastosuj częściowe poszukiwanie elementu wiodącego oraz skalowanie.

In [177]:
def LU_decomposition(M,n):
    L = [[0] * n for _ in range(n)] 
    U = [[0] * n for _ in range(n)]      
    
    for i in range(n): 
        for j in range(i, n):  #górny trójkąt
            sum = 0
            for k in range(i): 
                sum += L[i][k] * U[k][j]
            U[i][j] = M[i][j] - sum 

        for j in range(i, n): #dolny trójkąt
            L[i][i] = 1 
            if (i == j): continue
            sum = 0 
            for k in range(i): 
                sum += L[j][k] * U[k][i]
            L[j][i] = (M[j][i] - sum) / U[i][i]
            
    return L, U

In [194]:
A = np.random.randint(low=-10,high=10,size=(3,3)).astype(float)

L,U = LU_decomposition(A,n)
print("Macierz Wejściowa:")
for row in A: print(row)
    
print("\nL:")
for row in L: print(row)
    
print("\nU")
for row in U: print(row)
    
print("\nL * U:")
for row in np.dot(L,U): print(row)

Macierz Wejściowa:
[ 9.  1. -4.]
[8. 7. 0.]
[ 3.  3. -5.]

L:
[1, 0, 0]
[0.8888888888888888, 1, 0]
[0.3333333333333333, 0.43636363636363634, 1]

U
[9.0, 1.0, -4.0]
[0, 6.111111111111111, 3.5555555555555554]
[0, 0, -5.218181818181818]

L * U:
[ 9.  1. -4.]
[8. 7. 0.]
[ 3.  3. -5.]


### 2. Analiza obwodu elektrycznego
Napisz program, który:
 1. Wczytuje z pliku listę krawędzi grafu opisującego obwód elektryczny. Wagi krawędzi określają opór fragmentu obwodu między dwoma węzłami. Wierzchołki grafu identyfikowane są przez liczby naturalne.
 2. Wczytuje dodatkowo trójkę liczb $(s,t,E)$, przy czym para $(s,t)$ wskazuje między którymi węzłami sieci przyłożono siłę elektromotoryczną E. Opór wewnętrzny SEM można zaniedbać.
 3. Wykorzystując prawa Kirchhoffa znajduje natężenia prądu w każdej części obwodu i przedstawia je na rysunku w postaci grafu ważonego z etykietami.
 4. Wykorzystując metodę potencjałów węzłowych zaimplementuj alternatywne rozwiązanie problemu.
 5. Przedstaw (wizualizacja + sprawdzenie poprawności wyników) działanie programu dla grafów spójnych mających od 15 do 100 wierzchołków:
    - Spójny graf losowy
    - Graf 3-regularny (kubiczny)
    - Graf złożony z dwóch grafów losowych połączonych mostkiem 
    - Graf siatka 2D

In [ ]:
def drawGraph(G,maxI,pos):
    
    colorMap = plt.cm.cividis
    plotDPI = 600
    
    edgeArrowSize = 20
    edgeTextSize = 10
    
    vertexSize = 150
    vertexTextSize = 10
    
    
    nx.draw_networkx_nodes(G, pos, node_color = 'orange', node_size = vertexSize)
    nx.draw_networkx_labels(G, pos, font_size = vertexTextSize)

    colors = [I for _,I in nx.get_edge_attributes(G,'I').items()]
    nx.draw_networkx_edges(G, pos, edge_color=colors/maxI, edge_cmap=colorMap,
                           edge_vmin=0, edge_vmax=1, arrowsize=edgeArrowSize, node_size=vertexSize)

    
    edgeLabels = {e:'%.2f' %I for e,I in nx.get_edge_attributes(G,'I').items()}   
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edgeLabels, font_size = edgeTextSize)

    cm = plt.cm.ScalarMappable(cmap=colorMap , norm=plt.Normalize(0,maxI))
    plt.colorbar(cm)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    
    plt.show()